In [1894]:
import numpy as np
import pandas as pd

In [1895]:
#print('输出A文件路径',inputfile=input())

In [1896]:
"""
读取txt文件
该文本中的分割符既有空格又有制表符（‘/t’），sep参数用‘/s+’，可以匹配任何空格。
"""
#header=None:没有每列的column name，可以自己设定
#encoding='gb2312':其他编码中文显示错误
#sep=',':用逗号来分隔每行的数据
#index_col=0:设置第1列数据作为index
#index_col=0  指定第一列作为索引
inputfile='A53859-201905.TXT'

In [1897]:
data = pd.read_csv(inputfile,header=None,sep='/t',encoding='gb2312',index_col=0)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [1898]:
#提取数据
SJ=list(data.index)
#print(SJ)

In [1899]:
#SJ[0]为站点相关信息，进行提取
station=(SJ[0]).split(' ',-1)
jiben=['区站号','纬度','经度','观测场海波高度', '气压传感器海拔高度', '风速感应器距地（平台）高度',
         '观测平台距地高度','观测方式和测站类别', '观测项目标识' ,'质量控制指示码', '年份' ,'月份']
print('台站参数：')
df_station=pd.DataFrame(station,index=jiben,columns=['数据'])
print(df_station)

台站参数：
                                 数据
区站号                           53859
纬度                            3605N
经度                           11040E
观测场海波高度                      008513
气压传感器海拔高度                    008523
风速感应器距地（平台）高度                   102
观测平台距地高度                        000
观测方式和测站类别                       S12
观测项目标识         11111099110109111001
质量控制指示码                           1
年份                             2019
月份                               05


In [1900]:
'''
观测数据
每天4次定时观测时间分别为02、08、14、20时；
每天3次定时观测时间分别为08、14、20时；
每天24次定时观测时间分别为21时至20时，每1小时观测一次。
'''

'\n观测数据\n每天4次定时观测时间分别为02、08、14、20时；\n每天3次定时观测时间分别为08、14、20时；\n每天24次定时观测时间分别为21时至20时，每1小时观测一次。\n'

In [1901]:
'''
指示码：
P：本站气压
方位码（X）：
C：X=C 
    PC_1:本站气压每天24次定时值和日最高、最低值及出现时间共28组,分为两个记录
            第一个记录（21-08时）为12组，第二个记录（09-20时和最高及出现时间、最低值及出现时间）为16组
    PC_2:海平面气压每天4次定时值共4组。
单位：0.1hpa
'''
PC=SJ[SJ.index('PC'):SJ.index('TB')]
print(len(PC))
#通过等号，将数据划分成两部分
pc=[]
for i in range(len(PC)):
    if PC[i][-1]=='=':  # 若字符串最后一位为=
        pc=pc+[i]
print(pc)

94
[62, 93]


In [1902]:
#PC_1 第一组
PC_1=PC[1:pc[0]+1]
print(len(PC_1))
#print(PC_1)

62


In [1903]:
#对 PC_1 内的数据进行拼接,生成pc_1
pc_1=[]  
for i in range(len(PC_1)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            pc_1=pc_1+[PC_1[i-2]+' '+PC_1[i-1]]
print(len(pc_1))
#print(pc_1)

31


In [1904]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
pc0=['=','.']
for i in range(len(pc_1)):
    if pc_1[i][-1] in pc0:
        pc_1[i]=pc_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        pc_1[i]=pc_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(pc_1)):
    pc_1[i]=pc_1[i].split()
print(len(pc_1))
#print(pc_1)

31


In [1905]:
#格式转换
for i in range(len(pc_1)):
    for j in range(len(pc_1[i])):
        if j == 25 or j==27:
            pc_1[i][j]='%.2f' %(int(pc_1[i][j])/100)
            pc_1[i][j]=pc_1[i][j].replace('.',':')
        else:
            pc_1[i][j]='%.2f' %(int(pc_1[i][j])/10)

In [1906]:
year=df_station.loc['年份'].values
month=df_station.loc['月份'].values
#print(month)
index=[]
for i in range(1,len(pc_1)+1):
    index=index+["{}年-{}月-{:0>2d}日".format(year[0],month[0],i)]
#print(index)

In [1907]:
#新建dataframe
hour_28=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时',
         '最大','时间','最小','时间',]
print('本站气压：')
df_PC_1=pd.DataFrame(data=pc_1,index=index,columns=hour_28)
print(df_PC_1)

本站气压：
                  21时     22时     23时     24时      1时      2时      3时      4时  \
2019年-05月-01日  918.50  919.00  919.00  918.90  918.60  918.40  919.30  919.20   
2019年-05月-02日  920.90  921.30  921.30  921.30  921.30  921.20  920.70  920.50   
2019年-05月-03日  917.70  917.80  918.00  918.90  918.90  918.60  918.30  918.00   
2019年-05月-04日  918.10  918.50  918.90  918.70  918.50  918.00  917.60  917.40   
2019年-05月-05日  917.10  917.70  917.80  918.40  918.50  918.20  918.10  917.90   
2019年-05月-06日  922.10  922.80  923.60  923.60  923.50  923.20  923.20  923.70   
2019年-05月-07日  923.10  923.30  923.10  923.20  922.90  922.50  921.70  921.70   
2019年-05月-08日  919.40  919.90  919.70  919.50  918.90  918.00  918.00  917.40   
2019年-05月-09日  917.70  918.00  918.10  918.20  917.70  917.60  917.30  917.10   
2019年-05月-10日  914.10  914.80  914.80  914.70  914.50  914.40  914.30  914.50   
2019年-05月-11日  915.00  915.90  916.40  916.40  916.40  916.40  916.30  916.40   
2019年-05月-12日  913.30 

In [1908]:
#PC_2 第二组
PC_2=PC[pc[0]+1:pc[1]+1]
print(len(PC_2))
#print(PC_2)

31


In [1909]:
#去掉.和对不同的=结尾进行分割
for i in range(len(PC_2)):
    if PC_2[i][-1] in pc0:
        PC_2[i]=PC_2[i].rstrip('.')  #去除.
        PC_2[i]=PC_2[i].rstrip('=')  #去除=

In [1910]:
#进行分割
for i in range(len(PC_2)):
    PC_2[i]=PC_2[i].split()
print(len(PC_2))
#print(PC_2)

31


In [1911]:
#新建dataframe
hour_4=['00时','06时','12时','18时', ]
print('海平面气压：')
df_PC_2=pd.DataFrame(data=PC_2,index=index,columns=hour_4)
print(df_PC_2)

海平面气压：
                00时   06时   12时   18时
2019年-05月-01日  0150  0182  0171  0180
2019年-05月-02日  0194  0202  0150  0144
2019年-05月-03日  0149  0158  0126  0146
2019年-05月-04日  0137  0151  0115  0115
2019年-05月-05日  0137  0155  0155  0183
2019年-05月-06日  0204  0236  0213  0207
2019年-05月-07日  0204  0207  0170  0166
2019年-05月-08日  0154  0135  0138  0139
2019年-05月-09日  0163  0163  0107  0102
2019年-05月-10日  0105  0119  0087  0098
2019年-05月-11日  0121  0127  0079  0075
2019年-05月-12日  0089  0141  0191  0168
2019年-05月-13日  0180  0177  0106  0089
2019年-05月-14日  0079  0085  0043  0067
2019年-05月-15日  0075  0104  0061  0070
2019年-05月-16日  0071  0076  0043  0071
2019年-05月-17日  0070  0077  0033  0028
2019年-05月-18日  0037  0053  0037  0048
2019年-05月-19日  0122  0165  0163  0184
2019年-05月-20日  0213  0219  0166  0131
2019年-05月-21日  0145  0146  0100  0095
2019年-05月-22日  0101  0120  0075  0062
2019年-05月-23日  0083  0089  0062  0038
2019年-05月-24日  0030  0039  9992  9999
2019年-05月-25日  0010  0016  9973  9985
2019年

In [1912]:
'''
T:气温
B:每天24次定时和日最高、最低值及出现时间共28组，分为2个记录，
    第1个记录（21～08时）为12组，第2个记录（09～20时和最高值及出现时间、最低值及出现时间）为16组。
单位：0.1摄氏度
'''
TB=SJ[SJ.index('TB'):SJ.index('IB')]
#print(TB)
print(len(TB))
print(TB[-1])

63
0207 0238 0261 0289 0290 0296 0299 0302 0288 0284 0271 0260 0308 1409 0115 0533=


In [1913]:
#对数据进行拼接
tb1=[]  
for i in range(len(TB)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            tb1=tb1+[TB[i-1]+' '+TB[i]]
#print(tb1[0][-1])
print(len(tb1))
#print(tb1)

31


In [1914]:
#去掉.和对不同的=结尾进行分割
tb0=['=','.']
tb2=[]
for i in range(len(tb1)):
    if tb1[i][-1] in pc0:
        tb1[i].rstrip('.')
        tb2=tb2+[tb1[i][:-1]]
for i in range(len(tb2)):
    tb2[i]=tb2[i].split()
print(len(tb2))

31


In [1915]:
#格式转换
for i in range(len(tb2)):
    for j in range(len(tb2[i])):
        if j == 25 or j==27:   #时间格式处理
            tb2[i][j]='%.2f' %(int(tb2[i][j])/100)
            tb2[i][j]=tb2[i][j].replace('.',':')
        else:
            tb2[i][j]=float('%.2f' %(float(tb2[i][j])/10))

In [1916]:
#新建dataframe
print('气温：')
df_TB=pd.DataFrame(data=tb2,index=index,columns=hour_28)
print(df_TB)

气温：
                21时   22时   23时   24时    1时    2时    3时    4时    5时    6时  \
2019年-05月-01日  14.7  12.9  11.9  11.7  10.9  10.3   8.2   7.9   7.0   6.3   
2019年-05月-02日  11.1  10.4   9.6   8.5   7.8   7.2   6.6   6.1   5.1   4.8   
2019年-05月-03日  15.1  14.2  13.3  12.4  12.2  11.8  11.9  12.0  11.9  11.5   
2019年-05月-04日  14.9  14.7  14.4  14.0  14.1  14.2  14.1  14.0  14.0  13.8   
2019年-05月-05日  18.7  17.5  16.9  15.8  15.4  15.2  15.0  14.9  14.3  14.4   
2019年-05月-06日  14.8  14.7  14.3  13.9  13.7  13.7  13.6  13.4  12.6  13.0   
2019年-05月-07日  13.0  12.3  11.8  11.2  10.8  10.6  10.1   9.7   9.0   9.3   
2019年-05月-08日  14.3  13.4  11.2  10.8  10.5  10.5  10.3  10.2  10.0   9.7   
2019年-05月-09日  10.1   9.4   8.9   7.9   7.4   6.6   6.2   5.9   5.2   5.1   
2019年-05月-10日  17.4  16.2  15.2  13.8  12.3  11.7  10.7   9.9   9.3   8.8   
2019年-05月-11日  17.9  15.8  14.5  13.3  12.4  12.0  11.1  10.5  10.1   9.7   
2019年-05月-12日  19.9  18.0  16.5  15.2  14.1  13.3  12.4  12.7  15.3  14.

In [1917]:
df_TB['总和']=df_TB['21时']+df_TB['22时']+df_TB['23时']+df_TB['24时']+df_TB['1时']+df_TB['2时']+\
            df_TB['3时']+df_TB['4时']+df_TB['5时']+df_TB['6时']+df_TB['7时']+df_TB['8时']+\
            df_TB['9时']+df_TB['10时']+df_TB['11时']+df_TB['12时']+df_TB['13时']+df_TB['14时']+\
            df_TB['15时']+df_TB['16时']+df_TB['17时']+df_TB['18时']+df_TB['19时']+df_TB['20时']
print(df_TB['总和'])

2019年-05月-01日    315.5
2019年-05月-02日    327.0
2019年-05月-03日    393.9
2019年-05月-04日    438.0
2019年-05月-05日    416.3
2019年-05月-06日    349.0
2019年-05月-07日    310.3
2019年-05月-08日    291.3
2019年-05月-09日    314.2
2019年-05月-10日    413.7
2019年-05月-11日    454.4
2019年-05月-12日    369.7
2019年-05月-13日    356.5
2019年-05月-14日    435.1
2019年-05月-15日    490.4
2019年-05月-16日    503.9
2019年-05月-17日    495.9
2019年-05月-18日    519.8
2019年-05月-19日    433.6
2019年-05月-20日    379.2
2019年-05月-21日    433.2
2019年-05月-22日    501.1
2019年-05月-23日    545.1
2019年-05月-24日    595.0
2019年-05月-25日    613.9
2019年-05月-26日    578.1
2019年-05月-27日    475.9
2019年-05月-28日    393.5
2019年-05月-29日    386.2
2019年-05月-30日    537.5
2019年-05月-31日    520.5
Name: 总和, dtype: float64


In [1918]:
df_TB['平均']=round(df_TB['总和']/24,1)
print(df_TB['平均'])

2019年-05月-01日    13.1
2019年-05月-02日    13.6
2019年-05月-03日    16.4
2019年-05月-04日    18.3
2019年-05月-05日    17.3
2019年-05月-06日    14.5
2019年-05月-07日    12.9
2019年-05月-08日    12.1
2019年-05月-09日    13.1
2019年-05月-10日    17.2
2019年-05月-11日    18.9
2019年-05月-12日    15.4
2019年-05月-13日    14.9
2019年-05月-14日    18.1
2019年-05月-15日    20.4
2019年-05月-16日    21.0
2019年-05月-17日    20.7
2019年-05月-18日    21.7
2019年-05月-19日    18.1
2019年-05月-20日    15.8
2019年-05月-21日    18.0
2019年-05月-22日    20.9
2019年-05月-23日    22.7
2019年-05月-24日    24.8
2019年-05月-25日    25.6
2019年-05月-26日    24.1
2019年-05月-27日    19.8
2019年-05月-28日    16.4
2019年-05月-29日    16.1
2019年-05月-30日    22.4
2019年-05月-31日    21.7
Name: 平均, dtype: float64


In [1919]:
print(df_TB.columns)
print("气温数据处理完毕")

Index(['21时', '22时', '23时', '24时', '1时', '2时', '3时', '4时', '5时', '6时', '7时',
       '8时', '9时', '10时', '11时', '12时', '13时', '14时', '15时', '16时', '17时',
       '18时', '19时', '20时', '最大', '时间', '最小', '时间', '总和', '平均'],
      dtype='object')
气温数据处理完毕


In [1920]:
'''
U:相对湿度
B:每天24次定时值和自动观测日最小值及出现时间共26组，分为2个记录，
    第1个记录（21～08时）为12组，
    第2个记录（09～20时和最小值及出现时间）为14组。 
'''
UB=SJ[SJ.index('UB'):SJ.index('N9')]
#print(UB)
print(len(UB))
#print(UB[-1])

63


In [1921]:
#对数据进行拼接
ub1=[]  
for i in range(len(UB)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            ub1=ub1+[UB[i-1]+' '+UB[i]]
#print(tb1[0][-1])
print(len(ub1))
#print(ub1)

31


In [1922]:
#去掉.和对不同的=结尾进行分割
ub0=['=','.']
ub2=[]
for i in range(len(ub1)):
    if ub1[i][-1] in ub0:
        ub1[i].rstrip('.')
        ub2=ub2+[ub1[i][:-1]]
#print(a2)
for i in range(len(ub2)):
    ub2[i]=ub2[i].split()
#print(ub2)
print(len(ub2))

31


In [1923]:
#格式转换
for i in range(len(ub2)):
    for j in range(len(ub2[i])):
        if j == 25:
            ub2[i][j]='%.2f' %(int(ub2[i][j])/100)
            ub2[i][j]=ub2[i][j].replace('.',':')
        else:
            if ub2[i][j]=='%%':
                ub2[i][j]=int(100)
            else:
                ub2[i][j]=int(ub2[i][j])

In [1924]:
#新建dataframe
print('相对湿度：')
hour_26=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时',
         '最小湿度','时间',]
df_UB=pd.DataFrame(data=ub2,index=index,columns=hour_26)
print(df_UB)

相对湿度：
               21时  22时  23时  24时  1时  2时  3时  4时  5时  6时  ...  13时  14时  15时  \
2019年-05月-01日   28   34   36   35  40  40  48  47  47  50  ...   25   25   28   
2019年-05月-02日   68   74   77   84  86  86  89  90  99  99  ...   35   31   28   
2019年-05月-03日   53   56   59   63  64  65  64  64  64  67  ...   30   30   32   
2019年-05月-04日   99   99   99   99  99  99  99  99  99  99  ...   43   42   37   
2019年-05月-05日   58   64   67   71  74  76  77  78  84  84  ...   48   72   76   
2019年-05月-06日   99   99   99   99  99  90  80  52  82  45  ...   36   42   40   
2019年-05月-07日   38   43   49   52  54  59  60  59  61  62  ...   34   31   26   
2019年-05月-08日   37   44   65   68  69  67  66  68  67  70  ...   58   56   54   
2019年-05月-09日   72   77   83   92  97  99  99  99  99  99  ...   38   36   36   
2019年-05月-10日   48   54   58   64  70  75  79  88  89  92  ...   28   22   23   
2019年-05月-11日   40   46   50   54  57  60  63  66  68  71  ...   15   21   21   
2019年-05月-12日   36   4

In [1925]:
df_UB['相对湿度总和']=df_UB['21时']+df_UB['22时']+df_UB['23时']+df_UB['24时']+df_UB['1时']+df_UB['2时']+\
            df_UB['3时']+df_UB['4时']+df_UB['5时']+df_UB['6时']+df_UB['7时']+df_UB['8时']+\
            df_UB['9时']+df_UB['10时']+df_UB['11时']+df_UB['12时']+df_UB['13时']+df_UB['14时']+\
            df_UB['15时']+df_UB['16时']+df_UB['17时']+df_UB['18时']+df_UB['19时']+df_UB['20时']
print(df_UB['相对湿度总和'])

2019年-05月-01日     869
2019年-05月-02日    1475
2019年-05月-03日    1277
2019年-05月-04日    1717
2019年-05月-05日    1714
2019年-05月-06日    1365
2019年-05月-07日    1039
2019年-05月-08日    1425
2019年-05月-09日    1615
2019年-05月-10日    1203
2019年-05月-11日     970
2019年-05月-12日     952
2019年-05月-13日     709
2019年-05月-14日    1030
2019年-05月-15日    1249
2019年-05月-16日     843
2019年-05月-17日    1106
2019年-05月-18日    1121
2019年-05月-19日     803
2019年-05月-20日     447
2019年-05月-21日     531
2019年-05月-22日     487
2019年-05月-23日     518
2019年-05月-24日     568
2019年-05月-25日     872
2019年-05月-26日     799
2019年-05月-27日     597
2019年-05月-28日    1474
2019年-05月-29日    1519
2019年-05月-30日     930
2019年-05月-31日     533
Name: 相对湿度总和, dtype: int64


In [1926]:
df_UB['平均相对湿度']=round(df_UB['相对湿度总和']/24,0)
print(df_UB['平均相对湿度'])

2019年-05月-01日    36.0
2019年-05月-02日    61.0
2019年-05月-03日    53.0
2019年-05月-04日    72.0
2019年-05月-05日    71.0
2019年-05月-06日    57.0
2019年-05月-07日    43.0
2019年-05月-08日    59.0
2019年-05月-09日    67.0
2019年-05月-10日    50.0
2019年-05月-11日    40.0
2019年-05月-12日    40.0
2019年-05月-13日    30.0
2019年-05月-14日    43.0
2019年-05月-15日    52.0
2019年-05月-16日    35.0
2019年-05月-17日    46.0
2019年-05月-18日    47.0
2019年-05月-19日    33.0
2019年-05月-20日    19.0
2019年-05月-21日    22.0
2019年-05月-22日    20.0
2019年-05月-23日    22.0
2019年-05月-24日    24.0
2019年-05月-25日    36.0
2019年-05月-26日    33.0
2019年-05月-27日    25.0
2019年-05月-28日    61.0
2019年-05月-29日    63.0
2019年-05月-30日    39.0
2019年-05月-31日    22.0
Name: 平均相对湿度, dtype: float64


In [1927]:
print(df_UB.columns)
print('相对湿度处理完毕')

Index(['21时', '22时', '23时', '24时', '1时', '2时', '3时', '4时', '5时', '6时', '7时',
       '8时', '9时', '10时', '11时', '12时', '13时', '14时', '15时', '16时', '17时',
       '18时', '19时', '20时', '最小湿度', '时间', '相对湿度总和', '平均相对湿度'],
      dtype='object')
相对湿度处理完毕


In [1928]:
'''
R:降水量
6:X=6。
    定时降水量段每天3组（20-8）、（8-20）、（20-20）；
    自记降水量段每天（21-20时）共24组，分为2个记录，每个记录为12组；
    降水上下连接值段每月3组。
    无降水量-0000，微量-,,,,
'''
R6=SJ[SJ.index('R6'):SJ.index('W0')]
#print(R6)
print(len(R6))

95


In [1929]:
#通过等号，将数据划分成三部分
r6=[]
for i in range(len(R6)):
    if R6[i][-1]=='=':  # 若字符串最后一位为=
        r6=r6+[i]
print(r6)

[31, 93, 94]


In [1930]:
#R6_1 定时降水量
R6_1=R6[1:r6[0]+1]
print(len(R6_1))
print(R6_1)

31
['0000 0000 0000', '0000 0000 0000', '0000 0008 0008', ',,,, 0000 ,,,,', '0000 0033 0033', '0000 0000 0000', '0000 0000 0000', ',,,, ,,,, ,,,,', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 ,,,, ,,,,', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0001 0000 0001', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000', '0039 0000 0039', '0000 0000 0000', '0000 0000 0000', '0000 0000 0000=']


In [1931]:
#去掉.和对不同的=结尾进行分割,在r61上直接进行处理
r60=['=','.']
for i in range(len(R6_1)):
    if R6_1[i][-1] in r60:
        R6_1[i]=R6_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        R6_1[i]=R6_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(R6_1)):
    R6_1[i]=R6_1[i].split()
#print(R6_1)
print(len(R6_1))

31


In [1932]:
#格式转换
for i in range(len(R6_1)):
    for j in range(len(R6_1[i])):
        if R6_1[i][j]!= ',,,,':
            R6_1[i][j]='%.1f' %(int(R6_1[i][j])/10)
        else:
            R6_1[i][j].replace(',,,,','0.0')

In [1933]:
#新建dataframe
print('定时降水量：')
hour_3=['20-8时','8-20时','20-20时']
df_R6_1=pd.DataFrame(data=R6_1,index=index,columns=hour_3)
print(df_R6_1)

定时降水量：
              20-8时 8-20时 20-20时
2019年-05月-01日   0.0   0.0    0.0
2019年-05月-02日   0.0   0.0    0.0
2019年-05月-03日   0.0   0.8    0.8
2019年-05月-04日  ,,,,   0.0   ,,,,
2019年-05月-05日   0.0   3.3    3.3
2019年-05月-06日   0.0   0.0    0.0
2019年-05月-07日   0.0   0.0    0.0
2019年-05月-08日  ,,,,  ,,,,   ,,,,
2019年-05月-09日   0.0   0.0    0.0
2019年-05月-10日   0.0   0.0    0.0
2019年-05月-11日   0.0   0.0    0.0
2019年-05月-12日   0.0  ,,,,   ,,,,
2019年-05月-13日   0.0   0.0    0.0
2019年-05月-14日   0.0   0.0    0.0
2019年-05月-15日   0.0   0.0    0.0
2019年-05月-16日   0.0   0.0    0.0
2019年-05月-17日   0.0   0.0    0.0
2019年-05月-18日   0.0   0.0    0.0
2019年-05月-19日   0.1   0.0    0.1
2019年-05月-20日   0.0   0.0    0.0
2019年-05月-21日   0.0   0.0    0.0
2019年-05月-22日   0.0   0.0    0.0
2019年-05月-23日   0.0   0.0    0.0
2019年-05月-24日   0.0   0.0    0.0
2019年-05月-25日   0.0   0.0    0.0
2019年-05月-26日   0.0   0.0    0.0
2019年-05月-27日   0.0   0.0    0.0
2019年-05月-28日   3.9   0.0    3.9
2019年-05月-29日   0.0   0.0    0.0
201

In [1934]:
r62=R6[r6[0]+1:r6[1]+1]
print(len(r62))
#print(r62)

62


In [1935]:
#对数据进行拼接
R6_2=[]
for i in range(len(r62)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            R6_2=R6_2+[r62[i-2]+' '+r62[i-1]]
print(len(R6_2))
#print(R6_2)

31


In [1936]:
#去掉.和对不同的=结尾进行分割
for i in range(len(R6_2)):
    if R6_2[i][-1] in r60:
        R6_2[i]=R6_2[i].rstrip('.')  #去除.
        R6_2[i]=R6_2[i].rstrip('=')  #去除=
print(len(R6_2))
#print(R6_2)

31


In [1937]:
for i in range(len(R6_2)):
    R6_2[i]=R6_2[i].split()
print(len(R6_2))
#print(R6_2)

31


In [1938]:
#格式转换
for i in range(len(R6_2)):
    for j in range(len(R6_2[i])):
        if R6_2[i][j]!= ',,,,':
            R6_2[i][j]=float('%.1f' %(int(R6_2[i][j])/10))
        else:
            R6_2[i][j].replace(',,,,','0.0')

In [1939]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('定时降水量：')
df_R6_2=pd.DataFrame(data=R6_2,index=index,columns=hour_24)
print(df_R6_2)

定时降水量：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-05月-01日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-02日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-03日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-04日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-05日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-06日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-07日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-08日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-09日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-10日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-11日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2019年-05月-12日  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ... 

In [1940]:
df_R6_2['总降水量']=df_R6_2['21时']+df_R6_2['22时']+df_R6_2['23时']+df_R6_2['24时']+\
                df_R6_2['1时']+df_R6_2['2时']+df_R6_2['3时']+df_R6_2['4时']+df_R6_2['5时']+df_R6_2['6时']+\
                df_R6_2['7时']+df_R6_2['8时']+df_R6_2['9时']+df_R6_2['10时']+df_R6_2['11时']+\
                df_R6_2['12时']+df_R6_2['13时']+df_R6_2['14时']+df_R6_2['15时']+df_R6_2['16时']+df_R6_2['17时']+\
                df_R6_2['18时']+df_R6_2['19时']+df_R6_2['20时']
print(df_R6_2['总降水量'])

2019年-05月-01日    0.0
2019年-05月-02日    0.0
2019年-05月-03日    0.8
2019年-05月-04日    0.0
2019年-05月-05日    3.3
2019年-05月-06日    0.0
2019年-05月-07日    0.0
2019年-05月-08日    0.0
2019年-05月-09日    0.0
2019年-05月-10日    0.0
2019年-05月-11日    0.0
2019年-05月-12日    0.0
2019年-05月-13日    0.0
2019年-05月-14日    0.0
2019年-05月-15日    0.0
2019年-05月-16日    0.0
2019年-05月-17日    0.0
2019年-05月-18日    0.0
2019年-05月-19日    0.1
2019年-05月-20日    0.0
2019年-05月-21日    0.0
2019年-05月-22日    0.0
2019年-05月-23日    0.0
2019年-05月-24日    0.0
2019年-05月-25日    0.0
2019年-05月-26日    0.0
2019年-05月-27日    0.0
2019年-05月-28日    3.9
2019年-05月-29日    0.0
2019年-05月-30日    0.0
2019年-05月-31日    0.0
Name: 总降水量, dtype: float64


In [1941]:
#r63占时不用
#r63=R6[r6[1]+1:r6[2]+1]
#print(len(r63))
#print(r63)

In [1942]:
print(df_R6_2.columns)
print("降水量数据处理完毕")

Index(['21时', '22时', '23时', '24时', '1时', '2时', '3时', '4时', '5时', '6时', '7时',
       '8时', '9时', '10时', '11时', '12时', '13时', '14时', '15时', '16时', '17时',
       '18时', '19时', '20时', '总降水量'],
      dtype='object')
降水量数据处理完毕


In [1943]:
'''
F:风
N:X=N。
    第1段和第2段，每段每天24次定时值共24组，分为4个记录，每个记录为6组；
    风向风速每组6位，第一段和第二段前3位为风向，后三位为风速，
    最大极大风前3位为风速、后三位为风向。
    第3段每天最大、极大风共4组，第2、4组为出现时间。 
风速单位:0.1m/s
风向单位：度
'''
FN=SJ[SJ.index('FN'):SJ.index('DB')]
print(len(FN))
fn=[]
for i in range(len(FN)):
    if FN[i][-1]=='=':  # 若字符串最后一位为=
        fn=fn+[i]
print(fn)

280
[124, 248, 279]


In [1944]:
#FN_1 第一段：两分钟平均风速风向
FN_1=FN[1:fn[0]+1]
print(len(FN_1))
#print(FN_1)

124


In [1945]:
#对 FN_1 内的数据进行拼接,生成  fn_1
fn_1=[]  
for i in range(len(FN_1)+1):
    if i != 0:
        if i%4==0:   #能被 4 整除
            fn_1=fn_1+[FN_1[i-4]+' '+FN_1[i-3]+' '+FN_1[i-2]+' '+FN_1[i-1]]
print(len(fn_1))
#print(fn_1)

31


In [1946]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
fn0=['=','.']
for i in range(len(fn_1)):
    if fn_1[i][-1] in pc0:
        fn_1[i]=fn_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        fn_1[i]=fn_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(fn_1)):
    fn_1[i]=fn_1[i].split()
print(len(fn_1))
#print(fn_1)

31


In [1947]:
#格式转换
fx=[]  #风向
fs=[]  #风速
for i in range(len(fn_1)):
    for j in range(len(fn_1[i])):
            fx=fx+[fn_1[i][j][:3]]
            fs=fs+[int(fn_1[i][j][3:])/10]

In [1948]:
fx_point=[]  #方位
fx_angle=[]  #中心角度
'''
风向按照度数进行划分，划分为16个风向

'''
for i in range(len(fx)):
    if fx[i]=='PPC':
        fx_point=fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        fx_angle=fx_angle+[0]
    else:
        if int(fx[i])>=0 and int(fx[i])<=11.25:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]
        elif int(fx[i])>=11.26 and int(fx[i])<=33.75:
            fx_point=fx_point+['NNE']
            fx_angle=fx_angle+[22.5]
        elif int(fx[i])>=33.76 and int(fx[i])<=56.25:
            fx_point=fx_point+['NE']
            fx_angle=fx_angle+[45]
        elif int(fx[i])>=56.26 and int(fx[i])<=78.75:
            fx_point=fx_point+['ENE']
            fx_angle=fx_angle+[67.5]
        elif int(fx[i])>=78.76 and int(fx[i])<=101.25:
            fx_point=fx_point+['E']
            fx_angle=fx_angle+[90]
        elif int(fx[i])>=101.26 and int(fx[i])<=123.75:
            fx_point=fx_point+['ESE']
            fx_angle=fx_angle+[112.5]
        if int(fx[i])>=123.76 and int(fx[i])<=146.25:
            fx_point=fx_point+['SE']
            fx_angle=fx_angle+[135]
        elif int(fx[i])>=146.26 and int(fx[i])<=168.75:
            fx_point=fx_point+['SSE']
            fx_angle=fx_angle+[157.5]
        elif int(fx[i])>=168.76 and int(fx[i])<=191.25:
            fx_point=fx_point+['S']
            fx_angle=fx_angle+[180]
        elif int(fx[i])>=191.26 and int(fx[i])<=213.75:
            fx_point=fx_point+['SSW']
            fx_angle=fx_angle+[202.5]
        elif int(fx[i])>=213.76 and int(fx[i])<=236.25:
            fx_point=fx_point+['SW']
            fx_angle=fx_angle+[225]
        elif int(fx[i])>=236.26 and int(fx[i])<=258.75:
            fx_point=fx_point+['WSW']
            fx_angle=fx_angle+[247.5]
        elif int(fx[i])>=258.76 and int(fx[i])<=281.25:
            fx_point=fx_point+['W']
            fx_angle=fx_angle+[270]
        elif int(fx[i])>=281.76 and int(fx[i])<=303.75:
            fx_point=fx_point+['WNW']
            fx_angle=fx_angle+[295.5]
        elif int(fx[i])>=303.76 and int(fx[i])<=326.25:
            fx_point=fx_point+['NW']
            fx_angle=fx_angle+[315]
        elif int(fx[i])>=326.26 and int(fx[i])<=348.75:
            fx_point=fx_point+['NNW']
            fx_angle=fx_angle+[337.5]
        elif int(fx[i])>=348.76 and int(fx[i])<=360:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]

In [1949]:
fx1_point=np.array(fx_point).reshape(-1,24) 
print(len(fx1_point))  #方位

31


In [1950]:
fx1_angle=np.array(fx_angle).reshape(-1,24) 
print(len(fx1_angle))

31


In [1951]:
fs1=np.array(fs).reshape(-1,24) 
print(len(fs1))

31


In [1952]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('2分钟平均风速：')
df_FN1_fs=pd.DataFrame(data=fs1,index=index,columns=hour_24)
print(df_FN1_fs)

2分钟平均风速：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-05月-01日  1.8  3.2  3.7  4.0  1.5  2.7  2.2  1.9  2.4  2.4  ...  3.5   
2019年-05月-02日  1.1  0.9  1.0  1.9  2.3  2.9  3.0  1.5  1.5  1.9  ...  1.7   
2019年-05月-03日  1.5  1.9  2.5  1.9  3.0  2.8  3.1  2.2  2.5  1.9  ...  4.5   
2019年-05月-04日  1.2  1.5  0.4  0.9  1.0  1.2  1.3  1.3  1.1  1.2  ...  2.9   
2019年-05月-05日  0.8  1.6  3.0  2.2  3.0  1.7  3.4  2.2  0.9  2.7  ...  1.3   
2019年-05月-06日  0.9  1.0  0.0  1.1  0.7  1.7  1.7  1.6  1.4  2.0  ...  3.8   
2019年-05月-07日  0.8  1.0  0.7  0.9  1.2  1.7  2.8  2.5  1.4  1.6  ...  1.6   
2019年-05月-08日  3.6  3.0  2.8  1.9  2.8  3.5  2.0  1.5  1.3  1.2  ...  3.0   
2019年-05月-09日  0.7  0.4  1.2  1.7  2.7  1.8  1.2  2.2  1.4  2.3  ...  1.7   
2019年-05月-10日  0.8  2.0  1.8  0.8  0.9  1.6  1.5  2.1  2.7  1.4  ...  2.8   
2019年-05月-11日  3.5  2.7  2.8  2.2  2.1  2.9  1.7  1.5  1.7  1.7  ...  2.7   
2019年-05月-12日  0.0  1.9  3.2  3.5  3.1  2.9  2.7  3.1  4.7  0.7  ..

In [1953]:
df_FN1_fs['2分钟定时风速']=df_FN1_fs['21时']+df_FN1_fs['22时']+df_FN1_fs['23时']+df_FN1_fs['24时']+\
            df_FN1_fs['1时']+df_FN1_fs['2时']+df_FN1_fs['3时']+df_FN1_fs['4时']+df_FN1_fs['5时']+\
            df_FN1_fs['6时']+df_FN1_fs['7时']+df_FN1_fs['8时']+df_FN1_fs['9时']+df_FN1_fs['10时']+\
            df_FN1_fs['11时']+df_FN1_fs['12时']+df_FN1_fs['13时']+df_FN1_fs['14时']+df_FN1_fs['15时']+\
            df_FN1_fs['16时']+df_FN1_fs['17时']+df_FN1_fs['18时']+df_FN1_fs['19时']+df_FN1_fs['20时']
print(df_FN1_fs['2分钟定时风速'])

2019年-05月-01日    72.8
2019年-05月-02日    56.3
2019年-05月-03日    70.3
2019年-05月-04日    44.0
2019年-05月-05日    51.3
2019年-05月-06日    54.4
2019年-05月-07日    46.3
2019年-05月-08日    54.9
2019年-05月-09日    41.8
2019年-05月-10日    56.7
2019年-05月-11日    58.6
2019年-05月-12日    70.7
2019年-05月-13日    57.2
2019年-05月-14日    59.0
2019年-05月-15日    60.9
2019年-05月-16日    69.6
2019年-05月-17日    53.3
2019年-05月-18日    60.9
2019年-05月-19日    94.7
2019年-05月-20日    51.4
2019年-05月-21日    66.4
2019年-05月-22日    58.3
2019年-05月-23日    75.6
2019年-05月-24日    62.6
2019年-05月-25日    56.6
2019年-05月-26日    82.9
2019年-05月-27日    65.9
2019年-05月-28日    50.4
2019年-05月-29日    58.6
2019年-05月-30日    61.5
2019年-05月-31日    57.1
Name: 2分钟定时风速, dtype: float64


In [1954]:
df_FN1_fs['2分钟定时风速平均']=round(df_FN1_fs['2分钟定时风速']/24,1)
print(df_FN1_fs['2分钟定时风速平均'])

2019年-05月-01日    3.0
2019年-05月-02日    2.3
2019年-05月-03日    2.9
2019年-05月-04日    1.8
2019年-05月-05日    2.1
2019年-05月-06日    2.3
2019年-05月-07日    1.9
2019年-05月-08日    2.3
2019年-05月-09日    1.7
2019年-05月-10日    2.4
2019年-05月-11日    2.4
2019年-05月-12日    2.9
2019年-05月-13日    2.4
2019年-05月-14日    2.5
2019年-05月-15日    2.5
2019年-05月-16日    2.9
2019年-05月-17日    2.2
2019年-05月-18日    2.5
2019年-05月-19日    3.9
2019年-05月-20日    2.1
2019年-05月-21日    2.8
2019年-05月-22日    2.4
2019年-05月-23日    3.2
2019年-05月-24日    2.6
2019年-05月-25日    2.4
2019年-05月-26日    3.5
2019年-05月-27日    2.7
2019年-05月-28日    2.1
2019年-05月-29日    2.4
2019年-05月-30日    2.6
2019年-05月-31日    2.4
Name: 2分钟定时风速平均, dtype: float64


In [1955]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('2分钟风速风向：')
df_FN1_fx_point=pd.DataFrame(data=fx1_point,index=index,columns=hour_24)
print(df_FN1_fx_point)

2分钟风速风向：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-05月-01日  ENE   NE   NE  ENE    E  ENE    E   NE  ENE  ENE  ...    S   
2019年-05月-02日   SW  ESE   NE  NNE   NE   NE   NE  ENE    E  ESE  ...  WSW   
2019年-05月-03日  ESE   NE  ENE    E   NE    E  ENE  ENE  ENE    E  ...   SW   
2019年-05月-04日  ENE    E  SSE  WNW  NNE  ENE  ENE   NE   NE   NE  ...   SW   
2019年-05月-05日  ESE   NE  ENE    E  ENE  ENE  ENE  ENE  ESE  ENE  ...   SE   
2019年-05月-06日   NE  ENE  PPC   NE  ENE  NNE  ENE    E  ENE   NE  ...    E   
2019年-05月-07日  WNW    W    E   NE  ESE   NE   NE   NE  ESE   NE  ...    W   
2019年-05月-08日    E  ENE  NNE  ENE   NE   NE   NE  ENE    N  NNW  ...  WNW   
2019年-05月-09日  WSW  ENE  ENE   NE   NE    E  NNE   NE  ESE   NE  ...  SSW   
2019年-05月-10日  WSW   NE   NE    E   NE  ESE   NE  ENE   NE  NNE  ...  NNW   
2019年-05月-11日   NE  ENE  ENE    E  ENE   NE  ENE    E    E    E  ...  SSW   
2019年-05月-12日  PPC   NE  ENE  ENE  ENE  ENE  ENE  ENE   NE   NW  ..

In [1956]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('2分钟定时风速风向：')
df_FN1_fx_angle=pd.DataFrame(data=fx1_angle,index=index,columns=hour_24)
print(df_FN1_fx_angle)

2分钟定时风速风向：
                 21时    22时    23时    24时     1时     2时     3时     4时     5时  \
2019年-05月-01日   67.5   45.0   45.0   67.5   90.0   67.5   90.0   45.0   67.5   
2019年-05月-02日  225.0  112.5   45.0   22.5   45.0   45.0   45.0   67.5   90.0   
2019年-05月-03日  112.5   45.0   67.5   90.0   45.0   90.0   67.5   67.5   67.5   
2019年-05月-04日   67.5   90.0  157.5  295.5   22.5   67.5   67.5   45.0   45.0   
2019年-05月-05日  112.5   45.0   67.5   90.0   67.5   67.5   67.5   67.5  112.5   
2019年-05月-06日   45.0   67.5    0.0   45.0   67.5   22.5   67.5   90.0   67.5   
2019年-05月-07日  295.5  270.0   90.0   45.0  112.5   45.0   45.0   45.0  112.5   
2019年-05月-08日   90.0   67.5   22.5   67.5   45.0   45.0   45.0   67.5    0.0   
2019年-05月-09日  247.5   67.5   67.5   45.0   45.0   90.0   22.5   45.0  112.5   
2019年-05月-10日  247.5   45.0   45.0   90.0   45.0  112.5   45.0   67.5   45.0   
2019年-05月-11日   45.0   67.5   67.5   90.0   67.5   45.0   67.5   90.0   90.0   
2019年-05月-12日    0.0   45.0  

In [1957]:
#FN_2 第2段：十分钟分钟
FN_2=FN[fn[0]+1:fn[1]+1]
print(len(FN_2))
print(FN_2)

124
['061019 055031 052039 056037 091021 068024', '095019 052024 068024 064027 069023 064014', '110021 183027 187028 227035 255029 235052', '184052 202036 228039 228041 351016 178018.', '217012 106008 064009 035019 042025 041032', '039023 062018 097015 103018 115013 161012', '232025 241017 243022 261019 208026 240035', '192031 231044 224046 179035 226036 120008.', '096013 043019 066023 081018 044028 075025', '076027 062023 076023 084019 079016 078009', '236014 213027 240042 241055 239056 217043', '215040 226059 238029 232049 234011 120014.', '054016 062017 133008 280011 015005 048013', '062010 053013 048011 072009 052008 077014', '209017 205017 216019 222033 191018 278020', '130019 252037 220027 184041 216024 278012.', '114007 051018 055028 076020 064027 056020', '066033 063027 095009 047026 116006 207014', '284004 042016 010011 005012 056074 353015', '058015 088026 058052 231023 215014 002009.', '050012 070011 PPC000 036010 310007 038016', '073018 091016 060017 025016 061023 113023', 

In [1958]:
#对 FN_1 内的数据进行拼接,生成  fn_1
fn_2=[]  
for i in range(len(FN_2)+1):
    if i != 0:
        if i%4==0:   #能被 4 整除
            fn_2=fn_2+[FN_2[i-4]+' '+FN_2[i-3]+' '+FN_2[i-2]+' '+FN_2[i-1]]
print(len(fn_2))
#print(fn_1)

31


In [1959]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
for i in range(len(fn_2)):
    if fn_2[i][-1] in fn0:
        fn_2[i]=fn_2[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        fn_2[i]=fn_2[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(fn_2)):
    fn_2[i]=fn_2[i].split()
print(len(fn_2))
#print(fn_2)

31


In [1960]:
#格式转换
fx=[]  #风向
fs=[]  #风速
for i in range(len(fn_2)):
    for j in range(len(fn_2[i])):
            fx=fx+[fn_2[i][j][:3]]
            fs=fs+[int(fn_2[i][j][3:])/10]

In [1961]:
fx_point=[]  #方位
fx_angle=[]  #中心角度
'''
风向按照度数进行划分，划分为16个风向

'''
for i in range(len(fx)):
    if fx[i]=='PPC':
        fx_point=fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        fx_angle=fx_angle+[0]
    else:
        if int(fx[i])>=0 and int(fx[i])<=11.25:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]
        elif int(fx[i])>=11.26 and int(fx[i])<=33.75:
            fx_point=fx_point+['NNE']
            fx_angle=fx_angle+[22.5]
        elif int(fx[i])>=33.76 and int(fx[i])<=56.25:
            fx_point=fx_point+['NE']
            fx_angle=fx_angle+[45]
        elif int(fx[i])>=56.26 and int(fx[i])<=78.75:
            fx_point=fx_point+['ENE']
            fx_angle=fx_angle+[67.5]
        elif int(fx[i])>=78.76 and int(fx[i])<=101.25:
            fx_point=fx_point+['E']
            fx_angle=fx_angle+[90]
        elif int(fx[i])>=101.26 and int(fx[i])<=123.75:
            fx_point=fx_point+['ESE']
            fx_angle=fx_angle+[112.5]
        if int(fx[i])>=123.76 and int(fx[i])<=146.25:
            fx_point=fx_point+['SE']
            fx_angle=fx_angle+[135]
        elif int(fx[i])>=146.26 and int(fx[i])<=168.75:
            fx_point=fx_point+['SSE']
            fx_angle=fx_angle+[157.5]
        elif int(fx[i])>=168.76 and int(fx[i])<=191.25:
            fx_point=fx_point+['S']
            fx_angle=fx_angle+[180]
        elif int(fx[i])>=191.26 and int(fx[i])<=213.75:
            fx_point=fx_point+['SSW']
            fx_angle=fx_angle+[202.5]
        elif int(fx[i])>=213.76 and int(fx[i])<=236.25:
            fx_point=fx_point+['SW']
            fx_angle=fx_angle+[225]
        elif int(fx[i])>=236.26 and int(fx[i])<=258.75:
            fx_point=fx_point+['WSW']
            fx_angle=fx_angle+[247.5]
        elif int(fx[i])>=258.76 and int(fx[i])<=281.25:
            fx_point=fx_point+['W']
            fx_angle=fx_angle+[270]
        elif int(fx[i])>=281.76 and int(fx[i])<=303.75:
            fx_point=fx_point+['WNW']
            fx_angle=fx_angle+[295.5]
        elif int(fx[i])>=303.76 and int(fx[i])<=326.25:
            fx_point=fx_point+['NW']
            fx_angle=fx_angle+[315]
        elif int(fx[i])>=326.26 and int(fx[i])<=348.75:
            fx_point=fx_point+['NNW']
            fx_angle=fx_angle+[337.5]
        elif int(fx[i])>=348.76 and int(fx[i])<=360:
            fx_point=fx_point+['N']
            fx_angle=fx_angle+[0]

In [1962]:
fx2_point=np.array(fx_point).reshape(-1,24) 
print(len(fx2_point))  #方位

31


In [1963]:
fx2_angle=np.array(fx_angle).reshape(-1,24) 
print(len(fx2_angle))

31


In [1964]:
fs2=np.array(fs).reshape(-1,24) 
print(len(fs2))

31


In [1965]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('十分钟风速风向：')
df_FN2_fs=pd.DataFrame(data=fs2,index=index,columns=hour_24)
print(df_FN2_fs)

十分钟风速风向：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-05月-01日  1.9  3.1  3.9  3.7  2.1  2.4  1.9  2.4  2.4  2.7  ...  2.8   
2019年-05月-02日  1.2  0.8  0.9  1.9  2.5  3.2  2.3  1.8  1.5  1.8  ...  2.2   
2019年-05月-03日  1.3  1.9  2.3  1.8  2.8  2.5  2.7  2.3  2.3  1.9  ...  4.2   
2019年-05月-04日  1.6  1.7  0.8  1.1  0.5  1.3  1.0  1.3  1.1  0.9  ...  1.9   
2019年-05月-05日  0.7  1.8  2.8  2.0  2.7  2.0  3.3  2.7  0.9  2.6  ...  1.1   
2019年-05月-06日  1.2  1.1  0.0  1.0  0.7  1.6  1.8  1.6  1.7  1.6  ...  3.9   
2019年-05月-07日  0.8  0.6  0.5  0.9  1.2  1.9  2.7  2.4  1.4  1.7  ...  1.4   
2019年-05月-08日  2.5  3.8  2.6  1.9  2.8  3.2  1.4  2.1  1.5  1.1  ...  3.0   
2019年-05月-09日  0.3  0.3  1.5  1.2  2.6  1.6  1.7  2.2  1.4  2.2  ...  1.5   
2019年-05月-10日  0.5  1.9  2.1  1.3  1.3  1.6  1.6  2.0  2.3  1.8  ...  2.8   
2019年-05月-11日  3.2  2.7  2.7  2.2  2.3  2.8  1.4  1.3  2.1  1.6  ...  2.6   
2019年-05月-12日  0.3  1.7  2.9  3.1  3.1  3.4  2.5  2.8  4.2  0.9  ..

In [1966]:
df_FN2_fs['十分钟定时风速']=df_FN2_fs['21时']+df_FN2_fs['22时']+df_FN2_fs['23时']+df_FN2_fs['24时']+\
            df_FN2_fs['1时']+df_FN2_fs['2时']+df_FN2_fs['3时']+df_FN2_fs['4时']+df_FN2_fs['5时']+\
            df_FN2_fs['6时']+df_FN2_fs['7时']+df_FN2_fs['8时']+df_FN2_fs['9时']+df_FN2_fs['10时']+\
            df_FN2_fs['11时']+df_FN2_fs['12时']+df_FN2_fs['13时']+df_FN2_fs['14时']+df_FN2_fs['15时']+\
            df_FN2_fs['16时']+df_FN2_fs['17时']+df_FN2_fs['18时']+df_FN2_fs['19时']+df_FN2_fs['20时']
print(df_FN2_fs['十分钟定时风速'])

2019年-05月-01日    69.6
2019年-05月-02日    54.8
2019年-05月-03日    68.2
2019年-05月-04日    41.9
2019年-05月-05日    50.6
2019年-05月-06日    54.0
2019年-05月-07日    45.3
2019年-05月-08日    55.1
2019年-05月-09日    44.2
2019年-05月-10日    56.2
2019年-05月-11日    61.4
2019年-05月-12日    70.7
2019年-05月-13日    56.8
2019年-05月-14日    62.0
2019年-05月-15日    60.2
2019年-05月-16日    67.9
2019年-05月-17日    59.1
2019年-05月-18日    62.2
2019年-05月-19日    96.4
2019年-05月-20日    48.2
2019年-05月-21日    62.7
2019年-05月-22日    54.8
2019年-05月-23日    79.2
2019年-05月-24日    68.2
2019年-05月-25日    55.0
2019年-05月-26日    85.1
2019年-05月-27日    63.1
2019年-05月-28日    53.3
2019年-05月-29日    60.0
2019年-05月-30日    55.8
2019年-05月-31日    58.9
Name: 十分钟定时风速, dtype: float64


In [1967]:
df_FN2_fs['十分钟定时风速日平均']=round(df_FN2_fs['十分钟定时风速']/24,1)
print(df_FN2_fs['十分钟定时风速日平均'])

2019年-05月-01日    2.9
2019年-05月-02日    2.3
2019年-05月-03日    2.8
2019年-05月-04日    1.7
2019年-05月-05日    2.1
2019年-05月-06日    2.2
2019年-05月-07日    1.9
2019年-05月-08日    2.3
2019年-05月-09日    1.8
2019年-05月-10日    2.3
2019年-05月-11日    2.6
2019年-05月-12日    2.9
2019年-05月-13日    2.4
2019年-05月-14日    2.6
2019年-05月-15日    2.5
2019年-05月-16日    2.8
2019年-05月-17日    2.5
2019年-05月-18日    2.6
2019年-05月-19日    4.0
2019年-05月-20日    2.0
2019年-05月-21日    2.6
2019年-05月-22日    2.3
2019年-05月-23日    3.3
2019年-05月-24日    2.8
2019年-05月-25日    2.3
2019年-05月-26日    3.5
2019年-05月-27日    2.6
2019年-05月-28日    2.2
2019年-05月-29日    2.5
2019年-05月-30日    2.3
2019年-05月-31日    2.5
Name: 十分钟定时风速日平均, dtype: float64


In [1968]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('十分钟定时风速风向：')
df_FN2_fx_point=pd.DataFrame(data=fx2_point,index=index,columns=hour_24)
print(df_FN2_fx_point)

十分钟定时风速风向：
               21时  22时  23时  24时   1时   2时   3时   4时   5时   6时  ...  11时  \
2019年-05月-01日  ENE   NE   NE   NE    E  ENE    E   NE  ENE  ENE  ...    S   
2019年-05月-02日   SW  ESE  ENE   NE   NE   NE   NE  ENE    E  ESE  ...  WSW   
2019年-05月-03日    E   NE  ENE    E   NE  ENE  ENE  ENE  ENE    E  ...  WSW   
2019年-05月-04日   NE  ENE   SE    W  NNE   NE  ENE   NE   NE  ENE  ...   SW   
2019年-05月-05日  ESE   NE   NE  ENE  ENE   NE  ENE  ENE    E   NE  ...    N   
2019年-05月-06日   NE  ENE  PPC   NE   NW   NE  ENE    E  ENE  NNE  ...  ESE   
2019年-05月-07日    W    W    E   NE    E   NE   NE   NE  ESE   NE  ...    W   
2019年-05月-08日    E  ENE    N   NE   NE   NE   NE  ENE  NNE  NNW  ...  WNW   
2019年-05月-09日  WSW   NE   NE  ENE   NE  ENE   NE   NE  ESE   NE  ...    S   
2019年-05月-10日    W   NE   NE  ENE   NE    E   NE  ENE  ENE   NE  ...  WNW   
2019年-05月-11日   NE  ENE  ENE    E  ENE   NE    E    E  ENE    E  ...  SSW   
2019年-05月-12日    N   NE  ENE  ENE  ENE  ENE  ENE    E   NE  ESE  

In [1969]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('十分钟定时风速风向：')
df_FN2_fx_angle=pd.DataFrame(data=fx2_angle,index=index,columns=hour_24)
print(df_FN2_fx_angle)

十分钟定时风速风向：
                 21时    22时    23时    24时     1时     2时     3时     4时     5时  \
2019年-05月-01日   67.5   45.0   45.0   45.0   90.0   67.5   90.0   45.0   67.5   
2019年-05月-02日  225.0  112.5   67.5   45.0   45.0   45.0   45.0   67.5   90.0   
2019年-05月-03日   90.0   45.0   67.5   90.0   45.0   67.5   67.5   67.5   67.5   
2019年-05月-04日   45.0   67.5  135.0  270.0   22.5   45.0   67.5   45.0   45.0   
2019年-05月-05日  112.5   45.0   45.0   67.5   67.5   45.0   67.5   67.5   90.0   
2019年-05月-06日   45.0   67.5    0.0   45.0  315.0   45.0   67.5   90.0   67.5   
2019年-05月-07日  270.0  270.0   90.0   45.0   90.0   45.0   45.0   45.0  112.5   
2019年-05月-08日   90.0   67.5    0.0   45.0   45.0   45.0   45.0   67.5   22.5   
2019年-05月-09日  247.5   45.0   45.0   67.5   45.0   67.5   45.0   45.0  112.5   
2019年-05月-10日  270.0   45.0   45.0   67.5   45.0   90.0   45.0   67.5   67.5   
2019年-05月-11日   45.0   67.5   67.5   90.0   67.5   45.0   90.0   90.0   67.5   
2019年-05月-12日    0.0   45.0  

In [1970]:
#FN_3 第3组   每天最大极大风及时间
FN_3=FN[fn[1]+1:fn[2]+1]
print(FN_3)

['060231 1734 095241 1724', '057231 1654 088187 1514', '070233 1615 123238 1513', '045171 1717 075275 1208', '083055 1254 127051 1246', '050245 1422 080232 1401', '054231 1444 078238 1602', '048334 1635 081283 1643', '056181 1817 087196 1801', '058329 1517 099264 1309', '057181 1404 100187 1403', '070005 0837 129349 0918', '051183 1426 082167 1407', '062221 1456 106241 1627', '066060 1229 147060 1453', '067223 1426 101221 1424', '061171 1619 098181 1602', '068238 1244 097241 1236', '081313 2313 162340 0910', '058270 1219 095275 1216', '060319 1501 099303 1453', '057252 1142 114258 1135', '076233 1156 128238 1257', '060184 1700 097142 1611', '057233 1415 094255 1449', '075338 1439 132320 1817', '059341 2002 109017 0908', '053175 0032 099196 0023', '071223 1620 097221 1613', '071328 1604 114312 1345', '060220 1431 102247 1424=']


In [1971]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
fn0=['=','.']
for i in range(len(FN_3)):
    if FN_3[i][-1] in fn0:
        FN_3[i]=FN_3[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        FN_3[i]=FN_3[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(FN_3)):
    FN_3[i]=FN_3[i].split()
print(len(FN_3))
#print(FN_3)

31


In [1972]:
zd_fx=[]  #日最大,前三位-风速，后三位-风向
zd_fs=[]
zd_time=[]
jd_fx=[]   #日极大,前三位-风速，后三位-风向
jd_fs=[]
jd_time=[]
for i in range(len(FN_3)):
    for j in range(len(FN_3[i])):
        if j==0:
            zd_fx=zd_fx+[FN_3[i][j][3:]] 
            zd_fs=zd_fs+[int(FN_3[i][j][:3])/10]
        elif j==1:
            zd_time=zd_time+[FN_3[i][j][:2]+':'+FN_3[i][j][2:]]
        elif j==2:
            jd_fx=jd_fx+[FN_3[i][j][3:]]
            jd_fs=jd_fs+[int(FN_3[i][j][:3])/10]
        elif j==3:
            jd_time=jd_time+[FN_3[i][j][:2]+':'+FN_3[i][j][2:]]
            

In [1973]:
print(zd_fx)  #风向
print(zd_fs)
print(zd_time)

['231', '231', '233', '171', '055', '245', '231', '334', '181', '329', '181', '005', '183', '221', '060', '223', '171', '238', '313', '270', '319', '252', '233', '184', '233', '338', '341', '175', '223', '328', '220']
[6.0, 5.7, 7.0, 4.5, 8.3, 5.0, 5.4, 4.8, 5.6, 5.8, 5.7, 7.0, 5.1, 6.2, 6.6, 6.7, 6.1, 6.8, 8.1, 5.8, 6.0, 5.7, 7.6, 6.0, 5.7, 7.5, 5.9, 5.3, 7.1, 7.1, 6.0]
['17:34', '16:54', '16:15', '17:17', '12:54', '14:22', '14:44', '16:35', '18:17', '15:17', '14:04', '08:37', '14:26', '14:56', '12:29', '14:26', '16:19', '12:44', '23:13', '12:19', '15:01', '11:42', '11:56', '17:00', '14:15', '14:39', '20:02', '00:32', '16:20', '16:04', '14:31']


In [1974]:
print(jd_fx)  #风向
print(jd_fs)
print(jd_time)

['241', '187', '238', '275', '051', '232', '238', '283', '196', '264', '187', '349', '167', '241', '060', '221', '181', '241', '340', '275', '303', '258', '238', '142', '255', '320', '017', '196', '221', '312', '247']
[9.5, 8.8, 12.3, 7.5, 12.7, 8.0, 7.8, 8.1, 8.7, 9.9, 10.0, 12.9, 8.2, 10.6, 14.7, 10.1, 9.8, 9.7, 16.2, 9.5, 9.9, 11.4, 12.8, 9.7, 9.4, 13.2, 10.9, 9.9, 9.7, 11.4, 10.2]
['17:24', '15:14', '15:13', '12:08', '12:46', '14:01', '16:02', '16:43', '18:01', '13:09', '14:03', '09:18', '14:07', '16:27', '14:53', '14:24', '16:02', '12:36', '09:10', '12:16', '14:53', '11:35', '12:57', '16:11', '14:49', '18:17', '09:08', '00:23', '16:13', '13:45', '14:24']


In [1975]:
'''
对最大风向进行处理
'''
zd_fx_point=[]
zd_fx_angle=[]
for i in range(len(zd_fx)):
    if zd_fx[i]=='PPC':
        zd_fx_point=zd_fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        zd_fx_angle=zd_fx_angle+[0]
    else:
        if int(zd_fx[i])>=0 and int(zd_fx[i])<=11.25:
            zd_fx_point=zd_fx_point+['N']
            zd_fx_angle=zd_fx_angle+[0]
        elif int(zd_fx[i])>=11.26 and int(zd_fx[i])<=33.75:
            zd_fx_point=zd_fx_point+['NNE']
            zd_fx_angle=zd_fx_angle+[22.5]
        elif int(zd_fx[i])>=33.76 and int(zd_fx[i])<=56.25:
            zd_fx_point=zd_fx_point+['NE']
            zd_fx_angle=zd_fx_angle+[45]
        elif int(zd_fx[i])>=56.26 and int(zd_fx[i])<=78.75:
            zd_fx_point=zd_fx_point+['ENE']
            zd_fx_angle=zd_fx_angle+[67.5]
        elif int(zd_fx[i])>=78.76 and int(zd_fx[i])<=101.25:
            zd_fx_point=zd_fx_point+['E']
            zd_fx_angle=zd_fx_angle+[90]
        elif int(zd_fx[i])>=101.26 and int(zd_fx[i])<=123.75:
            zd_fx_point=zd_fx_point+['ESE']
            zd_fx_angle=zd_fx_angle+[112.5]
        if int(zd_fx[i])>=123.76 and int(zd_fx[i])<=146.25:
            zd_fx_point=zd_fx_point+['SE']
            zd_fx_angle=zd_fx_angle+[135]
        elif int(zd_fx[i])>=146.26 and int(zd_fx[i])<=168.75:
            zd_fx_point=zd_fx_point+['SSE']
            zd_fx_angle=zd_fx_angle+[157.5]
        elif int(zd_fx[i])>=168.76 and int(zd_fx[i])<=191.25:
            zd_fx_point=zd_fx_point+['S']
            zd_fx_angle=zd_fx_angle+[180]
        elif int(zd_fx[i])>=191.26 and int(zd_fx[i])<=213.75:
            zd_fx_point=zd_fx_point+['SSW']
            zd_fx_angle=zd_fx_angle+[202.5]
        elif int(zd_fx[i])>=213.76 and int(zd_fx[i])<=236.25:
            zd_fx_point=zd_fx_point+['SW']
            zd_fx_angle=zd_fx_angle+[225]
        elif int(zd_fx[i])>=236.26 and int(zd_fx[i])<=258.75:
            zd_fx_point=zd_fx_point+['WSW']
            zd_fx_angle=zd_fx_angle+[247.5]
        elif int(zd_fx[i])>=258.76 and int(zd_fx[i])<=281.25:
            zd_fx_point=zd_fx_point+['W']
            zd_fx_angle=zd_fx_angle+[270]
        elif int(zd_fx[i])>=281.76 and int(zd_fx[i])<=303.75:
            zd_fx_point=zd_fx_point+['WNW']
            zd_fx_angle=zd_fx_angle+[295.5]
        elif int(zd_fx[i])>=303.76 and int(zd_fx[i])<=326.25:
            zd_fx_point=zd_fx_point+['NW']
            zd_fx_angle=zd_fx_angle+[315]
        elif int(zd_fx[i])>=326.26 and int(zd_fx[i])<=348.75:
            zd_fx_point=zd_fx_point+['NNW']
            zd_fx_angle=zd_fx_angle+[337.5]
        elif int(zd_fx[i])>=348.76 and int(zd_fx[i])<=360:
            zd_fx_point=zd_fx_point+['N']
            zd_fx_angle=zd_fx_angle+[0]

In [1976]:
'''
对极大风向进行处理
'''
jd_fx_point=[]
jd_fx_angle=[]
for i in range(len(jd_fx)):
    if jd_fx[i]=='PPC':
        jd_fx_point=jd_fx_point+['PPC']    #C-PPC 静风，风速小于0.2m/s 
        jd_fx_angle=jd_fx_angle+[0]
    else:
        if int(jd_fx[i])>=0 and int(jd_fx[i])<=11.25:
            jd_fx_point=jd_fx_point+['N']
            jd_fx_angle=jd_fx_angle+[0]
        elif int(jd_fx[i])>=11.26 and int(jd_fx[i])<=33.75:
            jd_fx_point=jd_fx_point+['NNE']
            jd_fx_angle=jd_fx_angle+[22.5]
        elif int(jd_fx[i])>=33.76 and int(jd_fx[i])<=56.25:
            jd_fx_point=jd_fx_point+['NE']
            jd_fx_angle=jd_fx_angle+[45]
        elif int(jd_fx[i])>=56.26 and int(jd_fx[i])<=78.75:
            jd_fx_point=jd_fx_point+['ENE']
            jd_fx_angle=jd_fx_angle+[67.5]
        elif int(jd_fx[i])>=78.76 and int(jd_fx[i])<=101.25:
            jd_fx_point=jd_fx_point+['E']
            jd_fx_angle=jd_fx_angle+[90]
        elif int(jd_fx[i])>=101.26 and int(jd_fx[i])<=123.75:
            jd_fx_point=jd_fx_point+['ESE']
            jd_fx_angle=jd_fx_angle+[112.5]
        if int(jd_fx[i])>=123.76 and int(jd_fx[i])<=146.25:
            jd_fx_point=jd_fx_point+['SE']
            jd_fx_angle=jd_fx_angle+[135]
        elif int(jd_fx[i])>=146.26 and int(jd_fx[i])<=168.75:
            jd_fx_point=jd_fx_point+['SSE']
            jd_fx_angle=jd_fx_angle+[157.5]
        elif int(jd_fx[i])>=168.76 and int(jd_fx[i])<=191.25:
            jd_fx_point=jd_fx_point+['S']
            jd_fx_angle=jd_fx_angle+[180]
        elif int(jd_fx[i])>=191.26 and int(jd_fx[i])<=213.75:
            jd_fx_point=jd_fx_point+['SSW']
            jd_fx_angle=jd_fx_angle+[202.5]
        elif int(jd_fx[i])>=213.76 and int(jd_fx[i])<=236.25:
            jd_fx_point=jd_fx_point+['SW']
            jd_fx_angle=jd_fx_angle+[225]
        elif int(jd_fx[i])>=236.26 and int(jd_fx[i])<=258.75:
            jd_fx_point=jd_fx_point+['WSW']
            jd_fx_angle=jd_fx_angle+[247.5]
        elif int(jd_fx[i])>=258.76 and int(jd_fx[i])<=281.25:
            jd_fx_point=jd_fx_point+['W']
            jd_fx_angle=jd_fx_angle+[270]
        elif int(jd_fx[i])>=281.76 and int(jd_fx[i])<=303.75:
            jd_fx_point=jd_fx_point+['WNW']
            jd_fx_angle=jd_fx_angle+[295.5]
        elif int(jd_fx[i])>=303.76 and int(jd_fx[i])<=326.25:
            jd_fx_point=jd_fx_point+['NW']
            jd_fx_angle=jd_fx_angle+[315]
        elif int(jd_fx[i])>=326.26 and int(jd_fx[i])<=348.75:
            jd_fx_point=jd_fx_point+['NNW']
            jd_fx_angle=jd_fx_angle+[337.5]
        elif int(jd_fx[i])>=348.76 and int(jd_fx[i])<=360:
            jd_fx_point=jd_fx_point+['N']
            jd_fx_angle=jd_fx_angle+[0]

In [1977]:
#新建dataframe
#hour_8=['日最大风速','日最大风速风向','日最大风速风向中心角度','时间',
#       '日极大风速','日极大风速风向','日极大风速风向中心角度','时间'
#       ]
data_ZJ={'日最大风速':zd_fs , '日最大风速风向':zd_fx_point , '日最大风速风向中心角度':zd_fx_angle, '最大时间':zd_time ,
       '日极大风速':jd_fs , '日极大风速风向':jd_fx_point , '日极大风速风向中心角度':zd_fx_angle , '极大时间':jd_time}
print('最大极大风：')
df_FN_ZJ=pd.DataFrame(data=data_ZJ,index=index)
print(df_FN_ZJ.columns)
print(df_FN_ZJ)

最大极大风：
Index(['日最大风速', '日最大风速风向', '日最大风速风向中心角度', '最大时间', '日极大风速', '日极大风速风向',
       '日极大风速风向中心角度', '极大时间'],
      dtype='object')
               日最大风速 日最大风速风向  日最大风速风向中心角度   最大时间  日极大风速 日极大风速风向  日极大风速风向中心角度  \
2019年-05月-01日    6.0      SW        225.0  17:34    9.5     WSW        225.0   
2019年-05月-02日    5.7      SW        225.0  16:54    8.8       S        225.0   
2019年-05月-03日    7.0      SW        225.0  16:15   12.3     WSW        225.0   
2019年-05月-04日    4.5       S        180.0  17:17    7.5       W        180.0   
2019年-05月-05日    8.3      NE         45.0  12:54   12.7      NE         45.0   
2019年-05月-06日    5.0     WSW        247.5  14:22    8.0      SW        247.5   
2019年-05月-07日    5.4      SW        225.0  14:44    7.8     WSW        225.0   
2019年-05月-08日    4.8     NNW        337.5  16:35    8.1     WNW        337.5   
2019年-05月-09日    5.6       S        180.0  18:17    8.7     SSW        180.0   
2019年-05月-10日    5.8     NNW        337.5  15:17    9.9       W       

In [1978]:
'''
D:浅层地温
B:X=B。
    0cm段每天24次定时和自动观测日最高、最低值及出现时间共28组，分为2个记录， 
    第1个记录（21～08时）为12组，第2个记录（09～20时和最高值及出现时间、最低值及出现时间）为16组；
    5、10、15、20、40cm段，每段每天24次定时值共24组，分为2个记录，每个记录12组。
单位：0.1摄氏度
'''
DB=SJ[SJ.index('DB'):SJ.index('KB')]
print(len(DB))
db=[]
for i in range(len(DB)):
    if DB[i][-1]=='=':  # 若字符串最后一位为=
        db=db+[i]
print(db)
#print(DB)

373
[62, 124, 186, 248, 310, 372]


In [1979]:
#DB_1 :24小时值和最小、最低值及时间
DB_1 =DB[1:db[0]+1]
print(len(DB_1))
#print(DB_1)

62


In [1980]:
#对 DB_1 内的数据进行拼接,生成db_1
db_1=[]  
for i in range(len(DB_1)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_1=db_1+[DB_1[i-2]+' '+DB_1[i-1]]
print(len(db_1))
#print(db_1)

31


In [1981]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
db0=['=','.']
for i in range(len(db_1)):
    if db_1[i][-1] in pc0:
        db_1[i]=db_1[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_1[i]=db_1[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_1)):
    db_1[i]=db_1[i].split()
print(len(db_1))
#print(db_1)

31


In [1982]:
#格式转换
for i in range(len(db_1)):
    for j in range(len(db_1[i])):
        if j == 25 or j==27:
            db_1[i][j]='%.2f' %(int(db_1[i][j])/100)
            db_1[i][j]=db_1[i][j].replace('.',':')
        else: 
            if db_1[i][j][0]=='-':  #如果是负数
                if db_1[i][j][1]=='0':
                    db_1[i][j]=db_1[i][j][0]+db_1[i][j][2]+'.'+db_1[i][j][3]
                else:
                    db_1[i][j]=db_1[i][j][0]+db_1[i][j][1]+db_1[i][j][2]+'.'+db_1[i][j][3]
                        
            else:  #如果是正数
                db_1[i][j]=int(db_1[i][j])/10
#print(db_1)                   

In [1983]:
#新建dataframe
hour_28=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时',
         '最大','时间','最小','时间',]
print('浅层地温：')
df_DB_1=pd.DataFrame(data=db_1,index=index,columns=hour_28)
print(df_DB_1)

浅层地温：
                21时   22时   23时   24时    1时    2时    3时    4时    5时    6时  \
2019年-05月-01日  12.4  11.2  10.6  10.6   9.1   8.0   6.8   6.0   5.2   5.4   
2019年-05月-02日  11.0  10.3   8.8   7.7   7.0   6.3   5.8   5.3   4.7   4.6   
2019年-05月-03日  14.1  13.0  12.1  12.2  11.5  12.1  12.0  11.7  11.1  10.7   
2019年-05月-04日  15.5  15.1  15.1  14.8  14.7  14.4  14.2  13.9  13.9  13.8   
2019年-05月-05日  17.5  16.7  16.3  15.3  15.0  14.9  14.6  14.4  14.0  14.0   
2019年-05月-06日  14.7  14.4  14.2  13.9  13.6  13.0  12.4  11.2  11.8  11.4   
2019年-05月-07日  12.7  12.2  11.6  11.2  10.4  10.0   9.3   9.0   9.1   9.1   
2019年-05月-08日  12.5  12.1  10.6  10.4  10.1   9.8   9.9   9.8   9.7   9.8   
2019年-05月-09日   9.7   8.6   8.0   7.2   6.7   6.1   5.7   5.3   4.8   5.0   
2019年-05月-10日  15.7  15.7  14.3  12.2  11.0  11.9   9.7   8.9   8.4   8.5   
2019年-05月-11日  15.9  14.1  13.1  12.3  11.8  11.5  10.8   9.9   9.2   9.7   
2019年-05月-12日  16.7  15.7  14.8  13.7  12.9  12.2  11.5  11.4  12.2  1

In [1984]:
#DB_5 :5cm地温
DB_5 =DB[db[0]+1:db[1]+1]
print(len(DB_5))
#print(DB_5)

62


In [1985]:
#对 DB_5 内的数据进行拼接,生成db_5
db_5=[]
for i in range(len(DB_5)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_5=db_5+[DB_5[i-2]+' '+DB_5[i-1]]
print(len(db_5))
#print(db_5)

31


In [1986]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
for i in range(len(db_5)):
    if db_5[i][-1] in db0:
        db_5[i]=db_5[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_5[i]=db_5[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_5)):
    db_5[i]=db_5[i].split()
print(len(db_5))
#print(db_5)

31


In [1987]:
#格式转换
for i in range(len(db_5)):
    for j in range(len(db_5[i])):
        if db_5[i][j][0]=='-':  #如果是负数
            if db_5[i][j][1]=='0':
                db_5[i][j]=float(db_5[i][j][0]+db_5[i][j][2]+'.'+db_5[i][j][3])
            else:
                db_5[i][j]=float(db_5[i][j][0]+db_5[i][j][1]+db_5[i][j][2]+'.'+db_5[i][j][3])
                        
        else:  #如果是正数
            db_5[i][j]=float(int(db_5[i][j])/10)
#print(db_5) 

In [1988]:
#新建dataframe
hour_24=['21时','22时','23时','24时', '1时','2时','3时','4时','5时','6时','7时','8时',
        '9时','10时','11时','12时','13时','14时','15时','16时','17时','18时','19时','20时']
print('5cm地温：')
df_DB_5=pd.DataFrame(data=db_5,index=index,columns=hour_24)
print(df_DB_5)

5cm地温：
                21时   22时   23时   24时    1时    2时    3时    4时    5时    6时  \
2019年-05月-01日  17.1  15.6  14.5  13.8  13.1  12.2  11.3  10.4   9.7   9.0   
2019年-05月-02日  13.6  12.9  12.2  11.3  10.6   9.9   9.3   8.8   8.3   7.9   
2019年-05月-03日  18.5  17.1  16.0  15.1  14.6  14.3  14.0  13.8  13.4  12.9   
2019年-05月-04日  17.7  17.1  16.7  16.4  16.1  15.8  15.6  15.3  15.1  14.9   
2019年-05月-05日  21.2  20.0  19.2  18.4  17.7  17.3  16.9  16.5  16.2  15.8   
2019年-05月-06日  16.6  16.2  15.9  15.6  15.3  14.8  14.5  13.7  13.5  13.3   
2019年-05月-07日  15.1  14.5  14.0  13.5  13.1  12.6  12.1  11.7  11.4  11.2   
2019年-05月-08日  14.6  14.1  13.5  12.9  12.5  12.2  11.9  11.8  11.6  11.5   
2019年-05月-09日  13.4  12.5  11.8  11.0  10.4   9.9   9.4   9.0   8.5   8.1   
2019年-05月-10日  18.9  17.9  17.2  16.1  15.0  14.5  13.8  13.0  12.3  11.9   
2019年-05月-11日  19.6  18.4  17.3  16.4  15.7  15.0  14.5  13.9  13.2  12.8   
2019年-05月-12日  21.6  20.1  19.0  18.0  17.2  16.4  15.7  15.2  14.9  

In [1989]:
df_DB_5['5cm地温日总和']=df_DB_5['21时']+df_DB_5['22时']+df_DB_5['23时']+df_DB_5['24时']+\
            df_DB_5['1时']+df_DB_5['2时']+df_DB_5['3时']+df_DB_5['4时']+df_DB_5['5时']+\
            df_DB_5['6时']+df_DB_5['7时']+df_DB_5['8时']+df_DB_5['9时']+df_DB_5['10时']+\
            df_DB_5['11时']+df_DB_5['12时']+df_DB_5['13时']+df_DB_5['14时']+df_DB_5['15时']+\
            df_DB_5['16时']+df_DB_5['17时']+df_DB_5['18时']+df_DB_5['19时']+df_DB_5['20时']
print(df_DB_5['5cm地温日总和'])

2019年-05月-01日    347.3
2019年-05月-02日    404.0
2019年-05月-03日    438.3
2019年-05月-04日    504.2
2019年-05月-05日    460.2
2019年-05月-06日    387.0
2019年-05月-07日    355.4
2019年-05月-08日    346.5
2019年-05月-09日    381.7
2019年-05月-10日    465.9
2019年-05月-11日    499.0
2019年-05月-12日    444.4
2019年-05月-13日    454.1
2019年-05月-14日    487.9
2019年-05月-15日    535.3
2019年-05月-16日    533.3
2019年-05月-17日    569.7
2019年-05月-18日    565.0
2019年-05月-19日    549.9
2019年-05月-20日    522.6
2019年-05月-21日    554.9
2019年-05月-22日    588.4
2019年-05月-23日    608.3
2019年-05月-24日    658.1
2019年-05月-25日    688.7
2019年-05月-26日    677.2
2019年-05月-27日    601.7
2019年-05月-28日    554.4
2019年-05月-29日    511.2
2019年-05月-30日    610.4
2019年-05月-31日    609.6
Name: 5cm地温日总和, dtype: float64


In [1990]:
df_DB_5['5cm地温日平均']=round(df_DB_5['5cm地温日总和']/24,1)
print(df_DB_5['5cm地温日平均'])

2019年-05月-01日    14.5
2019年-05月-02日    16.8
2019年-05月-03日    18.3
2019年-05月-04日    21.0
2019年-05月-05日    19.2
2019年-05月-06日    16.1
2019年-05月-07日    14.8
2019年-05月-08日    14.4
2019年-05月-09日    15.9
2019年-05月-10日    19.4
2019年-05月-11日    20.8
2019年-05月-12日    18.5
2019年-05月-13日    18.9
2019年-05月-14日    20.3
2019年-05月-15日    22.3
2019年-05月-16日    22.2
2019年-05月-17日    23.7
2019年-05月-18日    23.5
2019年-05月-19日    22.9
2019年-05月-20日    21.8
2019年-05月-21日    23.1
2019年-05月-22日    24.5
2019年-05月-23日    25.3
2019年-05月-24日    27.4
2019年-05月-25日    28.7
2019年-05月-26日    28.2
2019年-05月-27日    25.1
2019年-05月-28日    23.1
2019年-05月-29日    21.3
2019年-05月-30日    25.4
2019年-05月-31日    25.4
Name: 5cm地温日平均, dtype: float64


In [1991]:
#DB_10 :10cm地温
DB_10 =DB[db[1]+1:db[2]+1]
print(len(DB_10))
#print(DB_10)

62


In [1992]:
#对 DB_10 内的数据进行拼接,生成db_10
db_10=[]
for i in range(len(DB_10)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_10=db_10+[DB_10[i-2]+' '+DB_10[i-1]]
print(len(db_10))
#print(db_10)

31


In [1993]:
#去掉.和对不同的=结尾进行分割,在 db_10 上直接进行处理
for i in range(len(db_10)):
    if db_10[i][-1] in db0:
        db_10[i]=db_10[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_10[i]=db_10[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_10)):
    db_10[i]=db_10[i].split()
print(len(db_10))
#print(db_10)

31


In [1994]:
#格式转换
for i in range(len(db_10)):
    for j in range(len(db_10[i])):
        if db_10[i][j][0]=='-':  #如果是负数
            if db_10[i][j][1]=='0':
                db_10[i][j]=float(db_10[i][j][0]+db_10[i][j][2]+'.'+db_10[i][j][3])
            else:
                db_10[i][j]=float(db_10[i][j][0]+db_10[i][j][1]+db_10[i][j][2]+'.'+db_10[i][j][3])

        else:  #如果是正数
            db_10[i][j]=float(int(db_10[i][j])/10)
#print(db_10) 

In [1995]:
#新建dataframe
print('10cm地温：')
df_DB_10=pd.DataFrame(data=db_10,index=index,columns=hour_24)
print(df_DB_10)

10cm地温：
                21时   22时   23时   24时    1时    2时    3时    4时    5时    6时  \
2019年-05月-01日  19.0  17.9  16.9  16.0  15.4  14.7  14.0  13.3  12.6  12.0   
2019年-05月-02日  14.8  14.3  13.8  13.3  12.7  12.2  11.7  11.2  10.8  10.4   
2019年-05月-03日  19.8  18.7  17.8  16.9  16.3  15.8  15.4  15.1  14.8  14.4   
2019年-05月-04日  18.6  18.0  17.5  17.2  16.8  16.6  16.3  16.1  15.9  15.6   
2019年-05月-05日  22.2  21.2  20.4  19.7  19.1  18.6  18.1  17.7  17.4  17.0   
2019年-05月-06日  17.7  17.3  16.9  16.6  16.3  16.0  15.7  15.3  14.9  14.7   
2019年-05月-07日  16.3  15.8  15.4  15.0  14.6  14.2  13.8  13.5  13.1  12.8   
2019年-05月-08日  15.5  15.1  14.7  14.3  13.9  13.6  13.3  13.1  12.9  12.7   
2019年-05月-09日  15.0  14.3  13.7  13.1  12.6  12.1  11.6  11.2  10.8  10.4   
2019年-05月-10日  19.4  18.6  18.0  17.3  16.6  16.0  15.5  14.9  14.3  13.8   
2019年-05月-11日  20.6  19.7  18.8  18.0  17.4  16.7  16.2  15.8  15.2  14.7   
2019年-05月-12日  22.5  21.4  20.5  19.7  18.9  18.3  17.6  17.1  16.6 

In [1996]:
df_DB_10['10cm地温日总和']=df_DB_10['21时']+df_DB_10['22时']+df_DB_10['23时']+df_DB_10['24时']+\
            df_DB_10['1时']+df_DB_10['2时']+df_DB_10['3时']+df_DB_10['4时']+df_DB_10['5时']+\
            df_DB_10['6时']+df_DB_10['7时']+df_DB_10['8时']+df_DB_10['9时']+df_DB_10['10时']+\
            df_DB_10['11时']+df_DB_10['12时']+df_DB_10['13时']+df_DB_10['14时']+df_DB_10['15时']+\
            df_DB_10['16时']+df_DB_10['17时']+df_DB_10['18时']+df_DB_10['19时']+df_DB_10['20时']
print(df_DB_10['10cm地温日总和'])

2019年-05月-01日    362.8
2019年-05月-02日    378.4
2019年-05月-03日    426.5
2019年-05月-04日    469.4
2019年-05月-05日    459.9
2019年-05月-06日    390.8
2019年-05月-07日    360.1
2019年-05月-08日    348.6
2019年-05月-09日    362.9
2019年-05月-10日    439.0
2019年-05月-11日    470.0
2019年-05月-12日    445.5
2019年-05月-13日    432.1
2019年-05月-14日    465.8
2019年-05月-15日    502.5
2019年-05月-16日    514.0
2019年-05月-17日    528.8
2019年-05月-18日    543.5
2019年-05月-19日    528.7
2019年-05月-20日    498.5
2019年-05月-21日    525.1
2019年-05月-22日    549.7
2019年-05月-23日    574.0
2019年-05月-24日    608.0
2019年-05月-25日    638.6
2019年-05月-26日    641.5
2019年-05月-27日    585.9
2019年-05月-28日    549.0
2019年-05月-29日    508.5
2019年-05月-30日    572.8
2019年-05月-31日    583.6
Name: 10cm地温日总和, dtype: float64


In [1997]:
df_DB_10['10cm地温日平均']=round(df_DB_10['10cm地温日总和']/24,1)
print(df_DB_10['10cm地温日平均'])

2019年-05月-01日    15.1
2019年-05月-02日    15.8
2019年-05月-03日    17.8
2019年-05月-04日    19.6
2019年-05月-05日    19.2
2019年-05月-06日    16.3
2019年-05月-07日    15.0
2019年-05月-08日    14.5
2019年-05月-09日    15.1
2019年-05月-10日    18.3
2019年-05月-11日    19.6
2019年-05月-12日    18.6
2019年-05月-13日    18.0
2019年-05月-14日    19.4
2019年-05月-15日    20.9
2019年-05月-16日    21.4
2019年-05月-17日    22.0
2019年-05月-18日    22.6
2019年-05月-19日    22.0
2019年-05月-20日    20.8
2019年-05月-21日    21.9
2019年-05月-22日    22.9
2019年-05月-23日    23.9
2019年-05月-24日    25.3
2019年-05月-25日    26.6
2019年-05月-26日    26.7
2019年-05月-27日    24.4
2019年-05月-28日    22.9
2019年-05月-29日    21.2
2019年-05月-30日    23.9
2019年-05月-31日    24.3
Name: 10cm地温日平均, dtype: float64


In [1998]:
#DB_15 :15cm地温
DB_15 =DB[db[2]+1:db[3]+1]
print(len(DB_15))
#print(DB_15)

62


In [1999]:
#对 DB_5 内的数据进行拼接,生成db_5
db_15=[]
for i in range(len(DB_15)+1):
    if i != 0:
        if i%2==0:   #能被2整除
            db_15=db_15+[DB_15[i-2]+' '+DB_15[i-1]]
print(len(db_15))
#print(db_15)

31


In [2000]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
for i in range(len(db_15)):
    if db_15[i][-1] in db0:
        db_15[i]=db_15[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        db_15[i]=db_15[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(db_15)):
    db_15[i]=db_15[i].split()
print(len(db_15))
#print(db_15)

31


In [2001]:
#格式转换
for i in range(len(db_15)):
    for j in range(len(db_15[i])):
        if db_15[i][j][0]=='-':  #如果是负数
            if db_15[i][j][1]=='0':
                db_15[i][j]=float(db_15[i][j][0]+db_15[i][j][2]+'.'+db_15[i][j][3])
            else:
                db_15[i][j]=float(db_15[i][j][0]+db_15[i][j][1]+db_15[i][j][2]+'.'+db_15[i][j][3])

        else:  #如果是正数
            db_15[i][j]=float(int(db_15[i][j])/10)
#print(db_15) 

In [2002]:
#新建dataframe
print('15cm地温：')
df_DB_15=pd.DataFrame(data=db_15,index=index,columns=hour_24)
print(df_DB_15)

15cm地温：
                21时   22时   23时   24时    1时    2时    3时    4时    5时    6时  \
2019年-05月-01日  19.4  18.7  18.0  17.4  16.8  16.3  15.8  15.3  14.7  14.2   
2019年-05月-02日  15.6  15.3  15.0  14.6  14.3  13.9  13.5  13.1  12.7  12.4   
2019年-05月-03日  19.8  19.1  18.5  17.9  17.4  16.9  16.5  16.2  15.9  15.6   
2019年-05月-04日  18.8  18.5  18.1  17.8  17.5  17.3  17.0  16.8  16.6  16.4   
2019年-05月-05日  22.0  21.5  20.9  20.4  19.9  19.5  19.1  18.7  18.4  18.0   
2019年-05月-06日  18.5  18.2  17.9  17.6  17.3  17.1  16.8  16.6  16.3  16.0   
2019年-05月-07日  17.1  16.7  16.4  16.1  15.8  15.5  15.2  14.9  14.6  14.4   
2019年-05月-08日  16.2  15.9  15.7  15.4  15.1  14.8  14.6  14.3  14.1  14.0   
2019年-05月-09日  15.7  15.3  14.9  14.5  14.1  13.8  13.4  13.0  12.7  12.4   
2019年-05月-10日  19.1  18.7  18.3  17.9  17.4  17.0  16.6  16.1  15.7  15.3   
2019年-05月-11日  20.5  20.0  19.5  18.9  18.4  17.9  17.5  17.1  16.7  16.3   
2019年-05月-12日  22.2  21.6  21.0  20.4  19.9  19.4  18.9  18.4  18.0 

In [2003]:
df_DB_15['15cm地温日总和']=df_DB_15['21时']+df_DB_15['22时']+df_DB_15['23时']+df_DB_15['24时']+\
            df_DB_15['1时']+df_DB_15['2时']+df_DB_15['3时']+df_DB_15['4时']+df_DB_15['5时']+\
            df_DB_15['6时']+df_DB_15['7时']+df_DB_15['8时']+df_DB_15['9时']+df_DB_15['10时']+\
            df_DB_15['11时']+df_DB_15['12时']+df_DB_15['13时']+df_DB_15['14时']+df_DB_15['15时']+\
            df_DB_15['16时']+df_DB_15['17时']+df_DB_15['18时']+df_DB_15['19时']+df_DB_15['20时']
print(df_DB_15['15cm地温日总和'])

2019年-05月-01日    378.8
2019年-05月-02日    369.9
2019年-05月-03日    422.2
2019年-05月-04日    450.4
2019年-05月-05日    463.3
2019年-05月-06日    403.0
2019年-05月-07日    372.7
2019年-05月-08日    359.5
2019年-05月-09日    359.7
2019年-05月-10日    426.4
2019年-05月-11日    456.5
2019年-05月-12日    451.4
2019年-05月-13日    427.4
2019年-05月-14日    459.1
2019年-05月-15日    488.8
2019年-05月-16日    507.9
2019年-05月-17日    511.9
2019年-05月-18日    536.5
2019年-05月-19日    523.6
2019年-05月-20日    495.1
2019年-05月-21日    516.2
2019年-05月-22日    536.2
2019年-05月-23日    562.2
2019年-05月-24日    588.0
2019年-05月-25日    617.4
2019年-05月-26日    628.8
2019年-05月-27日    586.6
2019年-05月-28日    554.5
2019年-05月-29日    515.5
2019年-05月-30日    558.2
2019年-05月-31日    576.0
Name: 15cm地温日总和, dtype: float64


In [2004]:
df_DB_15['15cm地温日平均']=round(df_DB_15['15cm地温日总和']/24,1)
print(df_DB_15['15cm地温日平均'])

2019年-05月-01日    15.8
2019年-05月-02日    15.4
2019年-05月-03日    17.6
2019年-05月-04日    18.8
2019年-05月-05日    19.3
2019年-05月-06日    16.8
2019年-05月-07日    15.5
2019年-05月-08日    15.0
2019年-05月-09日    15.0
2019年-05月-10日    17.8
2019年-05月-11日    19.0
2019年-05月-12日    18.8
2019年-05月-13日    17.8
2019年-05月-14日    19.1
2019年-05月-15日    20.4
2019年-05月-16日    21.2
2019年-05月-17日    21.3
2019年-05月-18日    22.4
2019年-05月-19日    21.8
2019年-05月-20日    20.6
2019年-05月-21日    21.5
2019年-05月-22日    22.3
2019年-05月-23日    23.4
2019年-05月-24日    24.5
2019年-05月-25日    25.7
2019年-05月-26日    26.2
2019年-05月-27日    24.4
2019年-05月-28日    23.1
2019年-05月-29日    21.5
2019年-05月-30日    23.3
2019年-05月-31日    24.0
Name: 15cm地温日平均, dtype: float64


In [2005]:
'''
S:日照时数
2:
    X=2。每天各时（3～21）日照时数共18组及日照总时数1组。
    X=A。每天各时（1～24）日照时数共24组及日出时间、日落时间、日照总时数各1组。
单位：0.1小时
'''
S2=SJ[SJ.index('S2')+1:SJ.index('BA')]
print(len(S2))
#print(S2)

31


In [2006]:
#去掉.和对不同的=结尾进行分割,在 pc_1 上直接进行处理
s20=['=','.']
for i in range(len(S2)):
    if S2[i][-1] in s20:
        S2[i]=S2[i].rstrip('.')  #删除 string 字符串末尾的指定字符
        S2[i]=S2[i].rstrip('=')  #删除 string 字符串末尾的指定字符
for i in range(len(S2)):
    S2[i]=S2[i].split()
print(len(S2))
#print(S2)

31


In [2007]:
#格式转换
for i in range(len(S2)):
    for j in range(len(S2[i])):
        if S2[i][j]!='NN':
            S2[i][j]=float(int(S2[i][j])/10)
        else:
            S2[i][j]='NN'
#print(S2) 

In [2008]:
#新建dataframe
hour_19=['3时','4时','5时','6时', '7时','8时','9时','10时','11时',
         '12时','13时','14时','15时','16时','17时','18时','19时','20时','日照总时数']
print('日照时数：')
df_S2=pd.DataFrame(data=S2,index=index,columns=hour_19)
print(df_S2)

日照时数：
               3时  4时   5时   6时   7时   8时   9时  10时  11时  12时  13时  14时  15时  \
2019年-05月-01日  NN  NN  0.2  1.0  0.9  0.6  0.6  0.0  0.0  0.0  0.0  0.0  0.0   
2019年-05月-02日  NN  NN  0.2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
2019年-05月-03日  NN  NN  0.0  0.2  0.3  0.7  0.3  0.2  0.9  0.2  0.0  0.0  0.0   
2019年-05月-04日  NN  NN  0.0  0.0  0.3  1.0  1.0  1.0  0.8  0.3  0.6  0.7  0.6   
2019年-05月-05日  NN  NN  0.0  0.0  0.1  0.5  0.7  0.1  0.2  0.0  0.0  0.0  0.0   
2019年-05月-06日  NN  NN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2019年-05月-07日  NN  NN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2019年-05月-08日  NN  NN  0.0  0.3  0.3  0.0  0.1  0.1  0.1  0.0  0.0  0.0  0.4   
2019年-05月-09日  NN  NN  0.0  0.0  0.0  0.1  1.0  1.0  1.0  0.2  0.0  0.6  1.0   
2019年-05月-10日  NN  NN  0.3  0.7  1.0  1.0  1.0  0.6  0.4  0.3  0.5  0.6  0.4   
2019年-05月-11日  NN  NN  0.1  0.7  1.0  1.0  1.0  1.0  0.9  0.7  0.5  0.7  0.8   
2019年-05月-12日  NN  NN  0.0  0.0  0

In [2009]:
filename='A'+station[0]+'-'+station[-2]+'-'+station[-1]+'.'+'xlsx'
print(filename)

A53859-2019-05.xlsx


In [2010]:
'''
保存，将多个保存在一个excel中

'''
from openpyxl import load_workbook
 
writer = pd.ExcelWriter(filename)

df_station.to_excel(excel_writer=writer,sheet_name="站点信息")

df_PC_1.to_excel(excel_writer=writer,sheet_name="本站气压")
df_PC_2.to_excel(excel_writer=writer,sheet_name="海平面气压")

df_TB.to_excel(excel_writer=writer,sheet_name="气温")

df_UB.to_excel(excel_writer=writer,sheet_name="相对湿度")

df_R6_1.to_excel(excel_writer=writer,sheet_name="定时降水量")
df_R6_2.to_excel(excel_writer=writer,sheet_name="降水量")

df_FN1_fs.to_excel(excel_writer=writer,sheet_name="2分钟定时风速") 
df_FN1_fx_point.to_excel(excel_writer=writer,sheet_name="2分钟定时风速风向方位")  
df_FN1_fx_angle.to_excel(excel_writer=writer,sheet_name="2分钟定时风速风向中心角度")  
df_FN2_fs.to_excel(excel_writer=writer,sheet_name="10分钟定时风速") 
df_FN2_fx_point.to_excel(excel_writer=writer,sheet_name="10分钟定时风速风向方位")  
df_FN2_fx_angle.to_excel(excel_writer=writer,sheet_name="10分钟定时风速风向中心角度")  
df_FN_ZJ.to_excel(excel_writer=writer,sheet_name="日最大极大风速风向") 

df_DB_1.to_excel(excel_writer=writer,sheet_name="浅层地温")
df_DB_5.to_excel(excel_writer=writer,sheet_name="5cm地温")
df_DB_10.to_excel(excel_writer=writer,sheet_name="10cm地温")
df_DB_15.to_excel(excel_writer=writer,sheet_name="15cm地温")

df_S2.to_excel(excel_writer=writer,sheet_name="日照时数")
#df_SA.to_excel(excel_writer=writer,sheet_name="日照时数")

writer.save()
writer.close()
print('导入excel成功')

导入excel成功
